In [3]:
#split
import os
import random
import shutil

# Define the paths for the input and output folders
input_folder = "/home/rubeena-sulthana/Downloads/AMERICAN GOLDFINCH"
output_folder_1 = "/home/rubeena-sulthana/Downloads/train"
output_folder_2 = "/home/rubeena-sulthana/Downloads/validation"
output_folder_3 = "/home/rubeena-sulthana/Downloads/test"

# Create the output folders if they don't exist
for folder in [output_folder_1, output_folder_2, output_folder_3]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Get the list of image files in the input folder
image_files = os.listdir(input_folder)

# Calculate the number of images for each category
total_images = len(image_files)
category_1_count = int(total_images * 0.8)
category_2_count = int(total_images * 0.1)
category_3_count = total_images - category_1_count - category_2_count

# Shuffle the image files randomly
random.shuffle(image_files)

# Move the images to the respective folders based on the distribution
for i, image_file in enumerate(image_files):
    image_path = os.path.join(input_folder, image_file)
    if i < category_1_count:
        output_path = os.path.join(output_folder_1, image_file)
    elif i < category_1_count + category_2_count:
        output_path = os.path.join(output_folder_2, image_file)
    else:
        output_path = os.path.join(output_folder_3, image_file)
    shutil.move(image_path, output_path)

In [16]:
##1. Implement a perceptron from scratcch 
import numpy as np
import os
from PIL import Image

# Define paths to your training, validation, and testing data
output_folder_1 = "/home/rubeena-sulthana/Downloads/train"
output_folder_2 = "/home/rubeena-sulthana/Downloads/test"
output_folder_3 = "/home/rubeena-sulthana/Downloads/validation/"

# Initialize perceptron parameters
learning_rate = 0.01
num_epochs = 2
input_shape = (28, 28)  # Assuming images are resized to 28x28 and grayscale

# Initialize perceptron weights and bias
num_inputs = np.prod(input_shape)
weights = np.zeros(num_inputs)
bias = 0.0

# Function to load data from folders
def load_data(folder_path, input_shape):
    X = []
    y = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):  # Assuming images are in JPG or PNG format
                img_path = os.path.join(root, file)
                label = 1 if "positive" in root else 0  # Example: folder structure decides the label
                with Image.open(img_path) as img:
                    img = img.convert('L')  # Convert to grayscale
                    img = img.resize(input_shape)  # Resize image to match input_shape
                    img_data = np.array(img).flatten()
                    X.append(img_data)
                y.append(label)
    X = np.array(X)
    y = np.array(y)
    return X, y

# Load training data
X_train, y_train = load_data(output_folder_1, input_shape)

# Training the perceptron
for epoch in range(num_epochs):
    total_loss = 0
    for i in range(len(X_train)):
        linear_output = np.dot(weights, X_train[i]) + bias
        prediction = 1 if linear_output >= 0 else 0
        error = y_train[i] - prediction
        total_loss += error ** 2  # Sum of squared errors (SSE)
        weights += learning_rate * error * X_train[i]
        bias += learning_rate * error

    # Calculate training accuracy and loss after each epoch
    correct_train = 0
    for i in range(len(X_train)):
        linear_output = np.dot(weights, X_train[i]) + bias
        prediction = 1 if linear_output >= 0 else 0
        if prediction == y_train[i]:
            correct_train += 1
    training_accuracy = correct_train / len(X_train)
    average_loss = total_loss / len(X_train)
    print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {training_accuracy * 100:.2f}%, Avg Loss: {average_loss:.2f}")

# Function to test accuracy on a dataset
def test_accuracy(X, y, weights, bias):
    correct = 0
    for i in range(len(X)):
        linear_output = np.dot(weights, X[i]) + bias
        prediction = 1 if linear_output >= 0 else 0
        if prediction == y[i]:
            correct += 1
    return correct / len(X)

# Load validation data
X_val, y_val = load_data(output_folder_2, input_shape)

# Validate and print accuracy
validation_accuracy = test_accuracy(X_val, y_val, weights, bias)
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")

# Load testing data
X_test, y_test = load_data(output_folder_3, input_shape)

# Test and print accuracy
test_accuracy = test_accuracy(X_test, y_test, weights, bias)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/2, Training Accuracy: 100.00%, Avg Loss: 0.01
Epoch 2/2, Training Accuracy: 100.00%, Avg Loss: 0.00
Validation Accuracy: 100.00%
Test Accuracy: 100.00%


In [17]:
##3Create a multi-layer perceptron (MLP) for digit classification (MNIST dataset)

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Define transforms for preprocessing
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load data from folders
train_data = datasets.MNIST('/home/rubeena-sulthana/Downloads/train', download=True, train=True, transform=transform)
validation_data = datasets.MNIST('/home/rubeena-sulthana/Downloads/validation', download=True, train=False, transform=transform)
test_data = datasets.MNIST('/home/rubeena-sulthana/Downloads/test', download=True, train=False, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

# Multi-layer perceptron (MLP) model
mlp = nn.Sequential(
    nn.Linear(784, 128),  # input layer (28x28) -> hidden layer (128)
    nn.ReLU(),
    nn.Linear(128, 64),  # hidden layer (128) -> hidden layer (64)
    nn.ReLU(),
    nn.Linear(64, 10)  # hidden layer (64) -> output layer (10)
)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.01, momentum=0.9)

# Train the network
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        optimizer.zero_grad()
        outputs = mlp(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/i}')

# Evaluate on training set
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Training Accuracy: {100 * correct / total} %')

# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Test Accuracy: {100 * correct / total} %')

# Evaluate on validation set
correct = 0
total = 0
with torch.no_grad():
    for data in validation_loader:
        inputs, labels = data
        inputs = inputs.view(-1, 784)
        outputs = mlp(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Validation Accuracy: {100 * correct / total} %')

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /home/rubeena-sulthana/Downloads/train/MNIST/raw/train-images-idx3-ubyte.gz to /home/rubeena-sulthana/Downloads/train/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /home/rubeena-sulthana/Downloads/train/MNIST/raw/train-labels-idx1-ubyte.gz to /home/rubeena-sulthana/Downloads/train/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

Extracting /home/rubeena-sulthana/Downloads/test/MNIST/raw/train-labels-idx1-ubyte.gz to /home/rubeena-sulthana/Downloads/test/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /home/rubeena-sulthana/Downloads/test/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/rubeena-sulthana/Downloads/test/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /home/rubeena-sulthana/Downloads/test/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/rubeena-sulthana/Downloads/test/MNIST/raw

Epoch 1, Loss: 0.4324938522745222
Epoch 2, Loss: 0.17937505245009952
Epoch 3, Loss: 0.13314065074681314
Epoch 4, Loss: 0.10452576792573312
Epoch 5, Loss: 0.08919755246027064
Epoch 6, Loss: 0.07545191423421807
Epoch 7, Loss: 0.0662390741503727
Epoch 8, Loss: 0.059288980182371595
Epoch 9, Loss: 0.05088042819563383
Epoch 10, Loss: 0.047420599705091375
Training Accuracy: 98.43833333333333 %
Test Accuracy: 97.31 %
Validation Accuracy: 97.31 %


In [21]:
##2Build and train a simple neural network using a framework like TensorFlow or PyTorch

import tensorflow as tf
import os
import numpy as np

# Define paths to your training, validation, and testing data
train_dir = "/home/rubeena-sulthana/Downloads/train"
test_dir = "/home/rubeena-sulthana/Downloads/test"
val_dir = "/home/rubeena-sulthana/Downloads/validation"

# Initialize perceptron parameters
learning_rate = 0.01
num_epochs = 10
input_shape = (28, 28, 3)  # Assuming images are resized to 28x28 and have 3 channels

# Function to load and preprocess data using TensorFlow
def load_data_tf(folder_path, input_shape):
    X = []
    y = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):  # Assuming images are in JPG or PNG format
                img_path = os.path.join(root, file)
                label = 1 if "positive" in root else 0  # Example: folder structure decides the label
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                X.append(img_array)
                y.append(label)
    X = np.array(X)
    y = np.array(y)
    return X, y

# Load training data
X_train, y_train = load_data_tf(train_dir, input_shape)

# Normalize pixel values to [0, 1]
X_train = X_train / 255.0

# Define the neural network architecture using TensorFlow's Sequential API
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification, so using sigmoid activation
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary crossentropy for binary classification
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=num_epochs)

# Function to evaluate accuracy on a dataset
def evaluate_accuracy(X, y):
    X = X / 255.0  # Normalize pixel values to [0, 1]
    _, accuracy = model.evaluate(X, y)
    return accuracy

# Evaluate training accuracy
train_accuracy = evaluate_accuracy(X_train, y_train)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

# Load validation data
X_val, y_val = load_data_tf(val_dir, input_shape)

# Evaluate validation accuracy
validation_accuracy = evaluate_accuracy(X_val, y_val)
print(f"Validation Accuracy: {validation_accuracy * 100:.2f}%")

# Load testing data
X_test, y_test = load_data_tf(test_dir, input_shape)

# Evaluate test accuracy
test_accuracy = evaluate_accuracy(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")



Epoch 1/10


/home/rubeena-sulthana/.local/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8653 - loss: 0.3293  
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.8068e-06 
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 2.1535e-08 
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 9.7900e-10 
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.3809e-10 
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.2820e-10 
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 5.9033e-11 
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 1.6277e-11 
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 2.1111e-11 
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 1.0000 - loss: 1.4875e-11 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.6015 
Training Accuracy: 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accu

In [22]:
##4Experiment with different regularization techniques on a neural network
import tensorflow as tf
import numpy as np
import os

# Assuming OpenCV is installed for image processing
train_dir = "/home/rubeena-sulthana/Downloads/train"
test_dir = "/home/rubeena-sulthana/Downloads/test"
val_dir = "/home/rubeena-sulthana/Downloads/validation"

# Initialize parameters
learning_rate = 0.01
num_epochs = 10
input_shape = (28, 28, 3)  # Assuming images are resized to 28x28 and have 3 channels

# Regularization parameters
l2_regularization = 0.001  # L2 regularization parameter
dropout_rate = 0.2  # Dropout rate
weight_decay = 0.0001  # Weight decay parameter

# Function to load and preprocess data using TensorFlow
def load_data_tf(folder_path, input_shape):
    X = []
    y = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".jpg") or file.endswith(".png"):  # Assuming images are in JPG or PNG format
                img_path = os.path.join(root, file)
                label = 1 if "positive" in root else 0  # Example: folder structure decides the label
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                X.append(img_array)
                y.append(label)
    X = np.array(X)
    y = np.array(y)
    return X, y

# Load training data
X_train, y_train = load_data_tf(train_dir, input_shape)
X_train = X_train / 255.0

# Load testing data
X_test, y_test = load_data_tf(test_dir, input_shape)
X_test = X_test / 255.0

# Load validation data
X_val, y_val = load_data_tf(val_dir, input_shape)
X_val = X_val / 255.0

# Define the neural network architecture for L2 regularization
model_l2 = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularization)),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_regularization)),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(l2_regularization))
])

# Compile the model for L2 regularization
model_l2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with L2 regularization
history_l2 = model_l2.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=1)

# Evaluate accuracy for L2 regularization on train data
l2_train_accuracy = model_l2.evaluate(X_train, y_train, verbose=0)[1]
print(f"L2 Regularization Train Accuracy: {l2_train_accuracy * 100:.2f}%")

# Evaluate accuracy for L2 regularization on validation data
l2_val_accuracy = model_l2.evaluate(X_val, y_val, verbose=0)[1]
print(f"L2 Regularization Validation Accuracy: {l2_val_accuracy * 100:.2f}%")

# Evaluate accuracy for L2 regularization on test data
l2_test_accuracy = model_l2.evaluate(X_test, y_test, verbose=0)[1]
print(f"L2 Regularization Test Accuracy: {l2_test_accuracy * 100:.2f}%")

# Define the neural network architecture for Dropout regularization
model_dropout = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model for Dropout regularization
model_dropout.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with Dropout regularization
history_dropout = model_dropout.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=1)

# Evaluate accuracy for Dropout regularization on train data
dropout_train_accuracy = model_dropout.evaluate(X_train, y_train, verbose=0)[1]
print(f"Dropout Train Accuracy: {dropout_train_accuracy * 100:.2f}%")

# Evaluate accuracy for Dropout regularization on validation data
dropout_val_accuracy = model_dropout.evaluate(X_val, y_val, verbose=0)[1]
print(f"Dropout Validation Accuracy: {dropout_val_accuracy * 100:.2f}%")

# Evaluate accuracy for Dropout regularization on test data
dropout_test_accuracy = model_dropout.evaluate(X_test, y_test, verbose=0)[1]
print(f"Dropout Test Accuracy: {dropout_test_accuracy * 100:.2f}%")

# Define the neural network architecture for Weight Decay regularization
model_weight_decay = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(weight_decay))
])

# Compile the model for Weight Decay regularization
optimizer_weight_decay = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model_weight_decay.compile(optimizer=optimizer_weight_decay, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with Weight Decay regularization
history_weight_decay = model_weight_decay.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=1)

# Evaluate accuracy for Weight Decay regularization on train data
weight_decay_train_accuracy = model_weight_decay.evaluate(X_train, y_train, verbose=0)[1]
print(f"Weight Decay Train Accuracy: {weight_decay_train_accuracy * 100:.2f}%")

# Evaluate accuracy for Weight Decay regularization on validation data
weight_decay_val_accuracy = model_weight_decay.evaluate(X_val, y_val, verbose=0)[1]
print(f"Weight Decay Validation Accuracy: {weight_decay_val_accuracy * 100:.2f}%")

# Evaluate accuracy for Weight Decay regularization on test data
weight_decay_test_accuracy = model_weight_decay.evaluate(X_test, y_test, verbose=0)[1]
print(f"Weight Decay Test Accuracy: {weight_decay_test_accuracy * 100:.2f}%")

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.5211 - loss: 0.9931 - val_accuracy: 1.0000 - val_loss: 0.3260
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.3254 - val_accuracy: 1.0000 - val_loss: 0.3186
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.3172 - val_accuracy: 1.0000 - val_loss: 0.3115
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.3100 - val_accuracy: 1.0000 - val_loss: 0.3041
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.3026 - val_accuracy: 1.0000 - val_loss: 0.2967
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.2952 - val_accuracy: 1.0000 - val_loss: 0.2895
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 1.0000 - loss: 0.2880 - val_accuracy: 1.0000 - val_loss: 0.2824
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 1.0000 - loss: 0.2810 - val_accuracy: 1.0000 - val_loss: 0.2755


In [23]:
##5Compare performance with various optimization algorithms
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
from torch.utils.data import DataLoader, TensorDataset
import os

# Define paths to datasets
training_path = "/home/rubeena-sulthana/Downloads/train"
validation_path = "/home/rubeena-sulthana/Downloads/validation"
testing_path = "/home/rubeena-sulthana/Downloads/test"

# Define transforms for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Paths to datasets
datasets = [
    {'name': 'train', 'path': training_path},
    {'name': 'validation', 'path': validation_path},
    {'name': 'test', 'path': testing_path}
]

# Initialize datasets and dataloaders
data_loaders = {}
for dataset in datasets:
    images = [f for f in os.listdir(dataset['path']) if f.endswith('.jpg') or f.endswith('.png')]
    data = []
    labels = []
    for img_name in images:
        img_path = os.path.join(dataset['path'], img_name)
        image = Image.open(img_path)
        image = transform(image)
        data.append(image)
        labels.append(torch.tensor(0))  # Assuming all labels are 0 for simplicity

    # Convert lists to tensors
    data_tensor = torch.stack(data)
    labels_tensor = torch.tensor(labels)

    # Create TensorDataset and DataLoader
    data_loaders[dataset['name']] = DataLoader(TensorDataset(data_tensor, labels_tensor), batch_size=64, shuffle=dataset['name'] == 'train')

# Neural network model
net = nn.Sequential(
    nn.Linear(224 * 224 * 3, 128),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
)

# Loss function
criterion = nn.CrossEntropyLoss()

# List of optimization algorithms to compare
optimizers = [
    {'name': 'SGD', 'optimizer': optim.SGD(net.parameters(), lr=0.01, momentum=0.9)},
    {'name': 'Adam', 'optimizer': optim.Adam(net.parameters(), lr=0.001)},
    {'name': 'RMSprop', 'optimizer': optim.RMSprop(net.parameters(), lr=0.001)},
    {'name': 'Adagrad', 'optimizer': optim.Adagrad(net.parameters(), lr=0.01)},
    {'name': 'Adadelta', 'optimizer': optim.Adadelta(net.parameters(), lr=1.0)}
]

# Training loop
for optimizer_info in optimizers:
    optimizer_name = optimizer_info['name']
    optimizer = optimizer_info['optimizer']
    print(f"Training with {optimizer_name} optimizer...")

    # Train the network
    num_epochs = 10
    for epoch in range(num_epochs):
        running_loss = 0.0
        net.train()  # Set the model to train mode
        for dataset_name in ['train', 'validation']:  # Include validation dataset for validation accuracy
            for i, (inputs, labels) in enumerate(data_loaders[dataset_name], 0):
                optimizer.zero_grad()
                inputs = inputs.view(-1, 224 * 224 * 3)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

        # Calculate training accuracy
        net.eval()  # Set the model to evaluation mode
        for dataset_name in ['train', 'validation']:
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in data_loaders[dataset_name]:
                    inputs = inputs.view(-1, 224 * 224 * 3)
                    outputs = net(inputs)
                    _, predicted = torch.max(outputs, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                accuracy = 100 * correct / total
                print(f'Epoch {epoch+1}, Dataset: {dataset_name.capitalize()}, Loss: {running_loss / len(data_loaders[dataset_name])}, Accuracy: {accuracy}%')

    # Evaluate on test set
    correct = 0
    total = 0
    net.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for inputs, labels in data_loaders['test']:
            inputs = inputs.view(-1, 224 * 224 * 3)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        test_accuracy = 100 * correct / total
    print(f'Test Accuracy with {optimizer_name} optimizer: {test_accuracy}%')


Training with SGD optimizer...
Epoch 1, Dataset: Train, Loss: 2.1376200318336487, Accuracy: 94.73684210526316%
Epoch 1, Dataset: Validation, Loss: 4.275240063667297, Accuracy: 85.71428571428571%
Epoch 2, Dataset: Train, Loss: 0.34521326422691345, Accuracy: 98.24561403508773%
Epoch 2, Dataset: Validation, Loss: 0.6904265284538269, Accuracy: 100.0%
Epoch 3, Dataset: Train, Loss: 0.11543199978768826, Accuracy: 100.0%
Epoch 3, Dataset: Validation, Loss: 0.2308639995753765, Accuracy: 100.0%
Epoch 4, Dataset: Train, Loss: 0.020172011601971462, Accuracy: 100.0%
Epoch 4, Dataset: Validation, Loss: 0.040344023203942925, Accuracy: 100.0%
Epoch 5, Dataset: Train, Loss: 0.0024842473483772665, Accuracy: 100.0%
Epoch 5, Dataset: Validation, Loss: 0.004968494696754533, Accuracy: 100.0%
Epoch 6, Dataset: Train, Loss: 0.00266523343151448, Accuracy: 100.0%
Epoch 6, Dataset: Validation, Loss: 0.00533046686302896, Accuracy: 100.0%
Epoch 7, Dataset: Train, Loss: 0.0019547336269170046, Accuracy: 100.0%
Epoc